<a href="https://colab.research.google.com/github/carlos-alves-one/-ML-Zoomcamp-Week-6/blob/main/ML_Zoomcamp_Week_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Goldsmiths University of London
**Author....: Carlos Manuel de Oliveira Alves**<br>
**Student..: cdeol003**<br>
**Created..: 10/10/2022**